# Evaluate the performance of a trained model

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
from torch.nn import DataParallel

from vos.models.STM import STM
from vos.algo.stm_train import STMAlgo
from vos.datasets.DAVIS import DAVIS_2017_TrainVal
from vos.utils.helpers import stack_images, stack_masks

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

Space-time Memory Networks: initialized.


In [2]:
davisroot = "/p300/videoObjSeg_dataset/DAVIS-2017-trainval-480p"
videod = DAVIS_2017_TrainVal(davisroot, mode= "val")
dataloader = DataLoader(videod)

In [3]:
statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200311/affine0.0-10.0-0.05-0.0/img_res-384,384/NNSTM/big_objects-True3/b_size-1/run_0"
state_dict = torch.load(os.path.join(statedict_root, "params.pkl"))
model_state_dict = state_dict['model_state_dict']

In [4]:
model = DataParallel(STM())
model.load_state_dict(model_state_dict)
model.cuda()
model.eval()

DataParallel(
  (module): STM(
    (Encoder_M): Encoder_M(
      (conv1_m): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv1_o): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (res2): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): C

In [5]:
algo = STMAlgo(
    eval_step_kwargs= dict(Mem_every= 5),
)
algo.initialize(model)

## Start evaluating and calculating the mean loss and performance

In [7]:
loss, IoU = 0.,0.
for i, data in enumerate(dataloader):
    print(f"Evaluating at itereation: {i}, loss: {loss/(i+1)}, IoU: {IoU/(i+1)}", end= "\r")
    eval_info, extra_info = algo.eval(i, data)
    loss += eval_info.loss
    IoU += eval_info.IoU